### Connect to mapr edge node and Import required libraries to get started ###

In [3]:
import pyspark

ModuleNotFoundError: No module named 'pyspark'

In [2]:
!pip3 install pyspark

zsh:1: /opt/homebrew/bin/pip3: bad interpreter: /opt/homebrew/opt/python@3.11/bin/python3.11: no such file or directory
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 316.9 MB 16.8 MB/s eta 0:00:01    |█████▎                          | 52.4 MB 13.9 MB/s eta 0:00:203.1 MB/s eta 0:00:20     |██████████                      | 99.1 MB 17.4 MB/s eta 0:00:13    | 156.2 MB 10.9 MB/s eta 0:00:157.6 MB/s eta 0:00:07     |████████████████████████▉       | 246.4 MB 11.1 MB/s eta 0:00:07███████████████████████▋      | 253.7 MB 14.9 MB/s eta 0:00:05     |██████████████████████████████▏ | 298.7 MB 10.4 MB/s eta 0:00:02�██████████ | 306.5 MB 11.6 MB/s eta 0:00:01
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425366 sha256=563bdc6b37f0ecab9a7f0ca4fbf5ecbaf27bfb34da80dd68bbfa4f025856c242
  Stored in directory: /Users/aagar19/Library/Caches/

In [3]:
!pip3 install pandas

zsh:1: /opt/homebrew/bin/pip3: bad interpreter: /opt/homebrew/opt/python@3.11/bin/python3.11: no such file or directory
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
from pyspark.conf import SparkConf 

In [ ]:
from pyspark.sql import HiveContext,SparkSession

In [15]:
spark = SparkSession.builder.master("rp000053440").getOrCreate()

In [16]:
conf = spark.sparkContext._conf.setAll([ \
        ('spark.cores.max', '4'), \
        ('spark.executor.memory', '4g'), \
        ('spark.driver.memory', '8g'), \
        ('spark.executor.cores', '4'), \
        ('spark.executor.instances', '6'), \
        ('spark.app.names', 'TextApp'), \
        ('hive.metastore.uris', 'thrift://obdplusr.optum.com:11000'), \
        ('packages' ,'io.delta:delta-core_2.12:2.0.0' ), \
        ('spark.yarn.queue' , 'fabbdprd_q1') \
                                        
                                       ])

In [17]:
spark.sparkContext.stop()

spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()

## ODIN Bronze layer Schema ##

###### All bronze layer Tables are available at below path: ######
    /datalake/optum/optuminsight/health_at_scale/hive/datafiles/DNHI/bronze/

##### READ ODIN dNHI diagCode file from bronze layer #####

In [25]:
read_ODIN_DiagCodes = spark.read.parquet("/datalake/optum/optuminsight/health_at_scale/hive/datafiles/DNHI/bronze/diagnosis_code")



In [26]:
read_ODIN_DiagCodes.printSchema()

root
 |-- DIAGNOSIS_KEY: decimal(38,0) (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DECM_CODE: string (nullable = true)
 |-- EFF_DATE: date (nullable = true)
 |-- END_DATE: date (nullable = true)
 |-- CODE_DESC: string (nullable = true)
 |-- MDC_CODE: string (nullable = true)
 |-- MDC_CODE_DESC: string (nullable = true)
 |-- GENDER_SPECIFIC_CODE: string (nullable = true)
 |-- CATEGORY_DTL_CODE: string (nullable = true)
 |-- CATEGORY_DTL_CODE_DESC: string (nullable = true)
 |-- CATEGORY_GENL_CODE: string (nullable = true)
 |-- CATEGORY_GENL_CODE_DESC: string (nullable = true)
 |-- UPDATE_DATE: date (nullable = true)
 |-- UPDATE_PROCESS_ID: string (nullable = true)
 |-- DIAGNOSIS_FIRST_3_CODE: string (nullable = true)
 |-- DIAGNOSIS_FIRST_3_DESC: string (nullable = true)
 |-- DIAGNOSIS_FIRST_4_CODE: string (nullable = true)
 |-- DIAGNOSIS_FIRST_4_DESC: string (nullable = true)
 |-- DIAGNOSIS_FULL_DESC: string (nullable = true)
 |-- DIAGNOSIS_LONG_DESC: string (nullable = tr

In [71]:
read_ODIN_DiagCodes.show(3,False)

+-----------------+-----+---------+----------+----------+---------------------------------+--------+-------------------------------------------------+--------------------+-----------------+----------------------------------------------+------------------+--------------------------+-----------+-----------------+----------------------+-----------------------------------+----------------------+-----------------------------------+----------------------------------------------------+---------------------------------------------+----------+
|DIAGNOSIS_KEY    |CODE |DECM_CODE|EFF_DATE  |END_DATE  |CODE_DESC                        |MDC_CODE|MDC_CODE_DESC                                    |GENDER_SPECIFIC_CODE|CATEGORY_DTL_CODE|CATEGORY_DTL_CODE_DESC                        |CATEGORY_GENL_CODE|CATEGORY_GENL_CODE_DESC   |UPDATE_DATE|UPDATE_PROCESS_ID|DIAGNOSIS_FIRST_3_CODE|DIAGNOSIS_FIRST_3_DESC             |DIAGNOSIS_FIRST_4_CODE|DIAGNOSIS_FIRST_4_DESC             |DIAGNOSIS_FULL_DESC          

##### READ ODIN dNHI file structure ###

In [30]:
read_ODIN_DiagCodes.select("DIAGNOSIS_KEY","MDC_CODE").show(5,False)

+-----------------+--------+
|DIAGNOSIS_KEY    |MDC_CODE|
+-----------------+--------+
|565551524940390  |09      |
|48575140939      |05      |
|51545453504297848|00      |
|50554854501      |00      |
|55565748514328148|00      |
+-----------------+--------+
only showing top 5 rows



## Kaplan Meier survival analysis on ODIN data ##

In [36]:
cancer_km = spark.read.parquet("/datalake/optum/optuminsight/health_at_scale/hive/datafiles/DNHI/bronze/km_analysis_breast_cancer")



In [38]:
cancer_km.printSchema()

root
 |-- NHI_INDIVIDUAL_ID: decimal(18,0) (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- YEAR_OF_BIRTH: integer (nullable = true)
 |-- STATE_CODE: string (nullable = true)
 |-- lastProcDate: date (nullable = true)
 |-- date_of_death: string (nullable = true)
 |-- MON_YR_DEATH: string (nullable = true)
 |-- firstDiagnosisDate: date (nullable = true)
 |-- memberAge: long (nullable = true)
 |-- range: string (nullable = true)
 |-- dateofDeath: date (nullable = true)
 |-- time: double (nullable = true)
 |-- event: integer (nullable = true)



In [ ]:
import matplotlib.pyplot as plt

from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()

In [39]:
df = cancer_kp.select("range","time","event")

In [ ]:
for group in groups:
    group_data = df[df['range'] == group]
    kmf.fit(group_data['time'],group_data['event'],label=group)
    kmf.plot()
    plt.legend(title = 'Group')
    plt.xlabel('Time')
    plt.ylabel('Survival Probability')

## Cohort of cancer drug adherence ##

In [75]:
ndc = spark.read.parquet("/datalake/optum/optuminsight/health_at_scale/hive/datafiles/DNHI/membership/member_aggregates/member_rx_claim")

In [ ]:
date_column='InvoiceDate'
id_column='memberId'
start_date='2022-01-01'
freq = 'MS' ## cohort gap period

In [76]:
ndc.printSchema()

root
 |-- memberId: decimal(18,0) (nullable = true)
 |-- ndcKey: decimal(38,0) (nullable = true)
 |-- firstFillDate: date (nullable = true)
 |-- lastFillDate: date (nullable = true)
 |-- refilCount: long (nullable = true)
 |-- memberAge: long (nullable = true)
 |-- sourceType: string (nullable = true)



In [81]:
ndc.show(10,False)

+-----------+-------------+-------------+------------+----------+---------+----------+
|memberId   |ndcKey       |firstFillDate|lastFillDate|refilCount|memberAge|sourceType|
+-----------+-------------+-------------+------------+----------+---------+----------+
|33003284687|678770413901 |2022-05-24   |2022-05-24  |1         |67       |rx_clm    |
|33003284687|7437279042375|2016-03-11   |2016-03-11  |1         |61       |rx_clm    |
|33003284687|7470681944587|2022-05-29   |2022-11-10  |3         |67       |rx_clm    |
|33003284687|7470681944937|2023-01-17   |2023-01-17  |1         |68       |rx_clm    |
|33003284687|7470689042375|2016-04-25   |2016-10-18  |3         |61       |rx_clm    |
|33003284687|7470689042739|2017-01-10   |2017-12-20  |9         |62       |rx_clm    |
|33003284687|7470689043110|2018-03-16   |2018-03-16  |1         |63       |rx_clm    |
|33003284687|7470689043187|2018-04-08   |2018-12-08  |4         |63       |rx_clm    |
|33003284687|7470689043474|2019-02-20   |20

In [82]:
ndc_cohort = ndc.filter("ndcKey == '7437279042375'")

In [83]:
ndc_cohort.count()

16339

In [106]:
ndc_cohort = ndc[['memberId','ndcKey', 'firstFillDate']].drop_duplicates()


In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from operator import attrgetter
import matplotlib.colors as mcolors

In [97]:
type(ndc_cohort)

pyspark.sql.dataframe.DataFrame

In [ ]:
ndc_cohort

In [103]:
med_key = ndc_cohort.select('ndcKey')


In [107]:
med_membrs = ndc_cohort.select('memberId')
